In [10]:
import pandas as pd
import numpy as np
import pickle as pkl

In [11]:
df1 = pd.read_csv('../../data1.csv')

In [12]:
df2 = pd.read_csv('../../data2.csv')

In [21]:
# Remove date overlap between the dataframes

df1['Date'] = pd.to_datetime(df1['Date'])
df2['Incident Date'] = pd.to_datetime(df2['Incident Date'])

# Get max date from df1 and min date from df2
max_date = df1['Date'].max()
print('Max date of df1:', max_date)
min_date = df2['Incident Date'].min()
print('Min date of df2:', min_date)

# Filter out rows in df1 where the Date is greater than or equal to min_date
df1 = df1[df1['Date'] < min_date]


Max date of df1: 2017-12-31 00:00:00
Min date of df2: 2018-01-01 00:00:00


In [22]:
# Find number of unique number of categories from the 'Category' column
print(df1['Category'].nunique())
print(df2['Incident Category'].nunique())

37
49


In [24]:
cols_to_keep_df1 = ['Category', 'Date', 'Time', 'PdDistrict', 'X', 'Y', 'Descript']
cols_to_keep_df2 = ['Incident Category','Incident Date','Incident Time','Police District','Latitude','Longitude','Incident Description']

In [25]:
df1_relevant = df1[cols_to_keep_df1]
df2_relevant = df2[cols_to_keep_df2]

In [26]:
category_mapping = {
    "Larceny Theft": "LARCENY/THEFT",
    "Recovered Vehicle": "RECOVERED VEHICLE",
    # "Lost Property": no corresponding category
    "Drug Violation": "DRUG/NARCOTIC",
    "Assault": "ASSAULT",
    "Malicious Mischief": "VANDALISM",
    "Non-Criminal": "NON-CRIMINAL",
    "Fraud": "FRAUD",
    "Warrant": "WARRANTS",
    "Other Offenses": "OTHER OFFENSES",
    "Robbery": "ROBBERY",
    # "Case Closure": no corresponding category
    "Stolen Property": "STOLEN PROPERTY",
    "Other Miscellaneous": "OTHER OFFENSES",
    "Other": "OTHER OFFENSES",
    # "Traffic Collision": no corresponding category
    "Suspicious Occ": "SUSPICIOUS OCC",
    "Disorderly Conduct": "DISORDERLY CONDUCT",
    "Weapons Carrying Etc": "WEAPON LAWS",
    "Rape": "SEX OFFENSES, FORCIBLE",
    "Drug Offense": "DRUG/NARCOTIC",
    "Missing Person": "MISSING PERSON",
    "Motor Vehicle Theft": "VEHICLE THEFT",
    "Burglary": "BURGLARY",
    "Fire Report": "ARSON",  # assuming these refer to arson-related incidents
    "Arson": "ARSON",
    "Vandalism": "VANDALISM",
    "Suicide": "SUICIDE",
    # "Traffic Violation Arrest": no corresponding category
    # "Courtesy Report": no corresponding category
    # "Offences Against The Family And Children": no corresponding category
    "Forgery And Counterfeiting": "FORGERY/COUNTERFEITING",
    # "Miscellaneous Investigation": no corresponding category
    "Sex Offense": "SEX OFFENSES, NON FORCIBLE",
    "Weapons Offense": "WEAPON LAWS",
    # "Vehicle Misplaced": no corresponding category
    "Suspicious": "SUSPICIOUS OCC",
    "Prostitution": "PROSTITUTION",
    # "Vehicle Impounded": no corresponding category
    "Embezzlement": "EMBEZZLEMENT",
    "Gambling": "GAMBLING",
    # "Homicide": no corresponding category
    # "Human Trafficking, Commercial Sex Acts": no corresponding category
    "Liquor Laws": "LIQUOR LAWS",
    # "Human Trafficking (A), Commercial Sex Acts": no corresponding category
    # "Civil Sidewalks": no corresponding category
    "Motor Vehicle Theft?": "VEHICLE THEFT",
    "Weapons Offence": "WEAPON LAWS",
    # "Human Trafficking (B), Involuntary Servitude": no corresponding category
}


In [27]:
# Assume df2_relevant has been processed as follows:
df2_relevant = df2[cols_to_keep_df2].copy()
df2_relevant['Incident Category'] = df2_relevant['Incident Category'].map(category_mapping)
df2_relevant = df2_relevant.dropna(subset=['Incident Category'])
valid_categories_df1 = df1['Category'].unique()
df2_relevant = df2_relevant[df2_relevant['Incident Category'].isin(valid_categories_df1)]
df2_relevant = df2_relevant.rename(columns={
    'Incident Category': 'Category',
    'Incident Date': 'Date',
    'Incident Time': 'Time',
    'Police District': 'PdDistrict',
    'Latitude': 'X',
    'Longitude': 'Y',
    'Incident Description': 'Descript'
})

# Identify the common categories between df1 and df2_relevant:
common_categories = set(df1['Category'].unique()).intersection(df2_relevant['Category'].unique())

# Filter df1 to only keep categories that are also in df2_relevant:
df1_relevant = df1[df1['Category'].isin(common_categories)]

# (Optional) Ensure df2_relevant also only has the common categories:
df2_relevant = df2_relevant[df2_relevant['Category'].isin(common_categories)]


In [28]:
# Categories from df1 that was not mapped to a category in df2
for cat in df1['Category'].unique():
    if cat not in df2_relevant['Category'].unique():
        print(cat)

TRESPASS
SECONDARY CODES
KIDNAPPING
DRUNKENNESS
BAD CHECKS
DRIVING UNDER THE INFLUENCE
LOITERING
BRIBERY
EXTORTION
PORNOGRAPHY/OBSCENE MAT
TREA


In [29]:
cols = ['Category', 'Date', 'Time', 'PdDistrict', 'X', 'Y', 'Descript']
df2_aligned = df2_relevant[cols]
df1_aligned = df1_relevant[cols]

In [30]:
# Convert df1_aligned['Date'] from 'MM/DD/YYYY' to datetime
df1_aligned['Date'] = pd.to_datetime(df1_aligned['Date'], format='%m/%d/%Y')

# Convert df2_aligned['Date'] from 'YYYY/MM/DD' to datetime
df2_aligned['Date'] = pd.to_datetime(df2_aligned['Date'], format='%Y/%m/%d')


/var/folders/sv/_w7_lr4s79bd9y8fpvjxy5d80000gn/T/ipykernel_62805/1577322905.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_aligned['Date'] = pd.to_datetime(df1_aligned['Date'], format='%m/%d/%Y')


In [31]:
# Concatenate both DataFrames
combined_df = pd.concat([df1_aligned, df2_aligned], ignore_index=True)

# Ensure consistent format (optional)
combined_df['Date'] = combined_df['Date'].dt.strftime('%Y-%m-%d')


In [32]:
# Merge Dataframes
combined_df = pd.concat([df1_aligned, df2_aligned], ignore_index=True)

In [33]:
len(combined_df["Category"].unique())

26

In [34]:
combined_df.to_pickle('combined_df.pkl')